In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from linearmodels.panel import RandomEffects, PanelOLS
import polars as pl

dr = DataReg()

In [ ]:
df_qcew = dr.regular_data()

df_qcew.group_by(["year", "qtr", "ein"]).agg(
            total_employment=pl.col("total_employment").sum(),
        )
tmp = dr.base_spatial_data().select(pl.col(["year","qtr","k_index"]))
tmp